<a href="https://colab.research.google.com/github/humanet1603/krx_using_api/blob/main/Mp3_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install vosk

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! pip install pydub


from vosk import Model, KaldiRecognizer
import os
from pydub import AudioSegment
import json

# MP3 파일을 WAV로 변환하는 함수
def mp3_to_wav(audio_file_path):
    sound = AudioSegment.from_mp3(audio_file_path)
    audio_file_path = audio_file_path.replace(".mp3", ".wav")
    sound.export(audio_file_path, format="wav")
    return audio_file_path

# WAV 파일을 텍스트로 변환하는 함수
def transcribe_audio(wav_file_path, model_path='vosk-model-small-en-us-0.15'):
    model = Model(model_path)
    rec = KaldiRecognizer(model, 16000)

    with open(wav_file_path, 'rb') as f:
        while True:
            data = f.read(4000)
            if len(data) == 0:
                break
            if rec.AcceptWaveform(data):
                pass

    final_result = json.loads(rec.FinalResult())
    return final_result['text']

# 전체 변환 과정 실행
def convert_mp3_to_txt(mp3_file_path, model_path):
    wav_file_path = mp3_to_wav(mp3_file_path)
    text = transcribe_audio(wav_file_path, model_path)
    os.remove(wav_file_path)  # 임시 WAV 파일 삭제
    return text

# 사용 예시
mp3_file_path = '/content/bandicam 2024-05-02 12-33-08-552.mp3'
model_path = 'model_path = r"C:\\vosk\\vosk-model-small-ko-0.22"'

text = convert_mp3_to_txt(mp3_file_path, model_path)
print(text)

#구글드라이브에서 제어하는 방법

## Google 드라이브 API와 관련 라이브러리 설치:

In [ ]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib


## Google 드라이브에서 파일을 다운로드하는 코드:

In [ ]:
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from google_auth_oauthlib.flow import InstalledAppFlow
import io

# 인증 및 Google 드라이브 서비스 생성
def create_service():
    flow = InstalledAppFlow.from_client_secrets_file(
        'client_secrets.json',  # Google Cloud에서 다운로드 받은 OAuth 2.0 클라이언트 ID JSON 파일
        scopes=['https://www.googleapis.com/auth/drive']
    )
    creds = flow.run_local_server(port=0)
    service = build('drive', 'v3', credentials=creds)
    return service

# Google 드라이브에서 파일 다운로드
def download_file(service, file_id, file_path):
    request = service.files().get_media(fileId=file_id)
    fh = io.FileIO(file_path, 'wb')
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        print("Download %d%%." % int(status.progress() * 100))

# 서비스 생성 및 파일 다운로드
service = create_service()
download_file(service, 'your_google_drive_file_id', 'local_path_to_save_model')

# 이제 모델 파일이 로컬에 저장되었으므로, vosk를 사용하여 음성 인식을 수행할 수 있습니다.
